<a href="https://colab.research.google.com/github/MaimomAlbert/Twitter-Sentiment_Analysis/blob/main/Data_cleaning_pre_processing_ipnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 16.2 MB/s eta 0:00:00


Data Cleaning/preprocessing


*   Removing stopword-a,the ,an ,is etc

*   Removing noise text(hastag,email,link,username) ,non-alphabets(digit,special character,punctuation mark),negation handeling(add neg)





In [ ]:
# utility
import pandas as pd
import numpy as np
import re
import contractions

# nltk
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/twitter_Sentiment_Analysis_MLP/training.1600000.processed.noemoticon.csv',engine='python',encoding='ISO-8859-1')
#define column names
df.columns=["label","id","date","query","user","text"]
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   label   1599999 non-null  int64 
 1   id      1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   query   1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
# Select only the needed columns
df = df[['label', 'text']]

# View the first 5 rows of the data
df.head()

,label,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [ ]:
#expanding the text /contarcting i.e can't to cannot
def expand_contractions(text: str):
  return contractions.fix(text)

In [ ]:
expand_contractions("can't")

'cannot'

The re.sub() function stands for a substring and returns a string with replaced values

In [ ]:
# removing noise text
def remove_noise_texts(text: str):
  # Patterns of the re  we want to remove
  email_re = r'\S+@\S+'
  mention_hashtag_re = r'(@|#)\S+'
  link_re = r'((www\.\S+)|(https?://\S+))|(\S+\.com)'
  repeating_suff_re   = r'(.)\1\1+'

  # Remove patterns in our textual data
  text = re.sub(repeating_suff_re, r'\1', text) # re.sub(pattern, replacement, string)= substituite and replace with the repeating stuff r'\1'
  text = re.sub(email_re, '', text) # replace with blank
  text = re.sub(mention_hashtag_re, '', text)
  text = re.sub(link_re, '', text)

  return text

In [ ]:
clean_text = remove_noise_texts("Check out this link: https://example.com and email me at example@example.com. Also, follow @user and use #hashtag!!!")
clean_text

'Check out this link:  and email me at  Also, follow  and use '

In [ ]:
# removing stopword
stop_words = stopwords.words("english")

def remove_stopwords(text: str, neg_keep: bool = False):
  # Remove 'not' 'no' from the list if neg is false. Otherwise keep the list as it is.nh
  stop_word_list = [word for word in stop_words if word not in ['not', 'no', 'nor']] if neg_keep else stop_words

  # Filter out words that are not in the list
  text = ' '.join([word for word in text.split() if word not in stop_word_list])

  return text

In [ ]:
remove_stopwords("This is a simple example no to demonstrate the function.", neg_keep=True)


'This simple example no demonstrate function.'

In [ ]:
# negation handeling
def negation_handle(text: str):
  text = re.sub(r'\b(?:not|no|never|cannot)\b\s+[a-z]+',
      lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG\2', match.group(0)),
      text)

  return text

In [ ]:
 negation_handle("I do not like this. She cannot see the problem. Never say never.")


'I do not NEGlike this. She cannot NEGsee the problem. Never say never.'

In [ ]:
# removing non-alphabets
def remove_non_alpha(text: str):
  # Pattern of the expression we want to remove
  non_alpha_digit_re = r'[^a-zA-Z]'

  # Remove patterns in our textual data
  text = re.sub(non_alpha_digit_re, r' ', text)

  return text

In [ ]:
remove_non_alpha("my name is 3000 albert . this is 1000== nice")

'my name is      albert   this is        nice'

In [ ]:
# remove redunant space
def remove_spaces(text: str):
  multi_spaces_re = r'\s\s+'
  text = re.sub(multi_spaces_re, ' ', text)
  return text




In [ ]:
remove_spaces("the   weather       is very     nice    ")

'the weather is very nice '

In [ ]:
# remove short word
def remove_short_words(text: str):
  text = ' '.join(word for word in text.split() if len(word) > 1)
  return text

In [ ]:
remove_short_words("the a is k l m n o p we not a word")

'the is we not word'

In [ ]:
#final cleaning of the data
def data_cleaning(text: str, handle_neg=False, remove_all_stopword=False):
  text = text.lower()
  text = expand_contractions(text)
  text = remove_noise_texts(text)
  if handle_neg:
    text = remove_stopwords(text, neg_keep=True)
    text = negation_handle(text)
  text = remove_non_alpha(text)
  text = remove_short_words(text)
  text = remove_spaces(text)
  if remove_all_stopword:
    text = remove_stopwords(text)

  return text.strip()

In [ ]:
cleaned_text = data_cleaning(
    "I can't believe it's not butter! Visit www.example.com for more details.",
    handle_neg=True,
    remove_all_stopword=True
)
cleaned_text


'cannot NEGbelieve NEGbutter visit details'

In [ ]:
# Apply cleaning function to text column (with negation)
df['neg_handled_text'] = df['text'].apply(lambda row: data_cleaning(row, handle_neg=True, remove_all_stopword=True))

# Apply cleaning function to text column (without negation)
df['normal_cleaned_text'] = df['text'].apply(data_cleaning,handle_neg=False,remove_all_stopword=True)

# Remove rows with no characters(with negation)
df_neg = df[df['neg_handled_text'] != ''][['neg_handled_text', 'label']]
#(without negation)
df_normal = df[df['normal_cleaned_text'] != ''][['normal_cleaned_text', 'label']]

In [ ]:
#initailizing
X_neg = df_neg['neg_handled_text'].values
y_neg = df_neg['label'].values
X_normal = df_normal['normal_cleaned_text'].values
y_normal = df_normal['label'].values
#saving the dataset
np.save('/content/drive/MyDrive/twitter_Sentiment_Analysis_MLP/dataset/feature_vectors_normal.npy', X_normal)
np.save('/content/drive/MyDrive/twitter_Sentiment_Analysis_MLP/dataset/target_vectors_normal.npy', y_normal)
np.save('/content/drive/MyDrive/twitter_Sentiment_Analysis_MLP/dataset/feature_vectors_neg.npy', X_neg)
np.save('/content/drive/MyDrive/twitter_Sentiment_Analysis_MLP/dataset/target_vectors_neg.npy', y_neg)

In [ ]:
len(X_neg[0])

79

In [ ]:
X_neg

array(['upset cannot NEGupdate facebook texting might cry result school today also blah',
       'dived many times ball managed save rest go bounds',
       'whole body feels itchy like fire', ...,
       'ready mojo makeover ask details',
       'happy th birthday boo al time tupac amaru shakur', 'happy'],
      dtype=object)

In [ ]:
type(X_neg)


numpy.ndarray